# Introduction to Data Science
## Homework 2

Student Name: Rafael Garcia Cano da Costa
NetID: rgc292
***

### Part 1: Case study
- Read [this article](http://www.nytimes.com/2012/02/19/magazine/shopping-habits.html) in the New York Times
- Use what we've learned in class as well as from the book and our readings to describe how one could set this up as a predictive modeling problem, such that they could have gotten the results that they did

Having in mind that the collection of customers' data offers the chance to better understand clients' wants and needs, and that the existence of a great change in people's lives is an opportunity to guide them towards new habits, Target have a good insight to increase its sales through gathering new customers to buy more items within its stores when experiencing the arrival of a new born. To accomplish this, the then current dataset containing clients' purchases could be used as a good start, but a continuous observation and colection of data referent to new purchases would be important in order to update the already established dataset. Having this done, the better approach would be the use of a supervised learning on a regression instance because the target attribute is known, which is the likelihood of a woman buyer be pregnant. Keeping this in mind, the exploration and cleaning of the data should be performed as a means to identify the best attributes to characterize the common purchase behavior a pregnant women presents. Therefore, labeling the ones offering the higher information gain would be valuable. After that, the decision tree would be a great predictive model to build a multivariate supervised segmentation.

### Part 2: Exploring data in the command line
For this part we will be using the data file located in `"data/advertising_events.csv"`. This file consists of records that pertain to some advertising events on a given day. There are 4 comma separated columns in this order: `userid`, `timestamp`, `domain`, and `action`. These fields are of type `int`, `int`, `string`, and `int` respectively. Answer the following questions using Linux/Unix bash commands. All questions can be answered in one line (sometimes, with pipes)! Some questions will have many possible solutions. Don't forget that in IPython notebooks you must prefix all bash commands with an exclamation point, i.e. `"!command arguments"`.

1\. How many records are in this file?

In [ ]:
!wc -l advertising_events.csv

2\. How many unique users are in this file?

In [ ]:
!cut -d "," -f 1 advertising_events.csv | sort | uniq | wc -l

3\. Rank all domains by the number of visits they received in descending order.

In [ ]:
!cut -d "," -f 3 advertising_events.csv | sort | uniq -c | sort -n -r

4\. List all records for the user with user id 37.

In [ ]:
!awk -F"," '$1 == 37 { print $1" "$2" "$3" "$4 }' advertising_events.csv

### Part 3: Dealing with messy data
Not all data you will deal with is going to be clean. In fact, much of it will be very messy! For example, we have the HTML page that lists the contributors to Facebook's [osquery](https://github.com/facebook/osquery) project that is hosted on [Github.com](https://github.com). In this case, all we are interested in are the contributors and how many commits each of them has. Given the HTML page in `"data/osquery_contributors.html"` you will sift through tons of irrelevant data so that you can build a useful data structure.

Notice that the first six (out of 59 total) contributors are named "theopolis", "marpaia", "javuto", "jedi22", "unixist", and "mofarrell". They have 553, 477, 104, 49, 30, 25 commits respectively.

![Screenshot](images/osquery_contributors.png)

To get a better of understanding of how this data is stored in the file, try searching through the raw data file for these usernames to look for any patterns. Your final dictionary should have 59 elements!

1\. Turn this data into a Python dictionary called `contributors` where the keys are the contributor names and the values are the number of commits that each contributor has.

In [ ]:
contributors = dict()
# Place your code here
!awk '/author=/' osquery_contributors.html | cut -d "=" -f 4 | cut -d '"' -f 1 > names.txt
!awk '/author=/' osquery_contributors.html | cut -d "=" -f 4 | cut -d '>' -f 2 | cut -d ' ' -f 1 > values.txt
!paste -d"," names.txt values.txt > dictpair.txt

file = open('dictpair.txt')
for line in file:
    x = line.split(",")    
    a=x[0]
    b=x[1].split("\n")[0]
    contributors[a]=b
# This line will print your dictionary for grading purposed. Do not remove this line!!!    
print contributors

### Part 4: Dealing with data Pythonically

In [ ]:
# You might find these packages useful. You may import any other you want!
import pandas as pd
import numpy as np

1\. Download the data set `"data/ads_dataset.tsv"` and load it into a Python Pandas data frame called `ads`.

In [ ]:
from pandas import DataFrame
ads = DataFrame.from_csv("ads_dataset.tsv", sep="\t")

2\. Write a Python function called `getDfSummary()` that does the following:
- Takes as input a data frame
- For each variable in the data frame calculates the following features:
  - `number_nan` to count the number of missing not-a-number values
  - Ignoring missing, NA, and Null values:
    - `number_distinct` to count the number of distinct values a variable can take on
    - `mean`, `max`, `min`, `std` (standard deviation), and `25%`, `50%`, `75%` to correspond to the appropriate percentiles
- All of these new features should be loaded in a new data frame. Each row of the data frame should be a variable from the input data frame, and the columns should be the new summary features.
- Returns this new data frame containing all of the summary information

Hint: The pandas `describe()` method returns a useful series of values that can be used here.

In [ ]:
#def getDfSummary(ads):
def getDfSummary(file):
# Place your code here
  number_nan = pd.DataFrame(file.isnull().sum(), columns=['number_nan'])
  description = pd.DataFrame(file.describe().T) 
  description_change  = description.rename(columns = {'count':'number_distinct'}) 
  union = number_nan.join(description_change)
  output_data = union
#  return output_data    
  return output_data

getDfSummary(ads)

3\. How long does it take for your `getDfSummary()` function to work on your `ads` data frame? Show us the results below.

Hint: `%timeit getDfSummary(ads)`

In [ ]:
%timeit getDfSummary(ads)

4\. Using the results returned from `getDfSummary()`, which fields, if any, contain missing `NaN` values?

In [ ]:
df = getDfSummary(ads)
df[df['number_nan'] == 0]

5\. For the fields with missing values, does it look like the data is missing at random? Are there any other fields that correlate perfectly, or predict that the data is missing? If missing, what should the data value be?

Hint: create another data frame that has just the records with a missing value. Get a summary of this data frame using `getDfSummary()` and compare the differences. Do some feature distributions change dramatically?

In [ ]:
missing_df = pd.DataFrame(ads[ads['buy_freq'] != 1])
getDfSummary(missing_df)

# It does not look that the data is missing. The fact that the respective value for "isbuyer" is zero explains the fact 
# that there is no buy_freq. If it was mising, the data should be zero. 

6\. Which variables are binary?

In [ ]:
#Although (dtypes) shows that no one of the variables are binary, the variable (isbuyer) seems to be used as a boolean: 
  # 1 as yes and 0 as no 
# no one of them
df1 = getDfSummary(ads).T
df1.dtypes
# no one of them
df2 = getDfSummary(ads)
df2.dtypes
# no one of them
ads.dtypes

7\. Generate a correlation matrix for the `ads` data frame. Is there any redundancy in the data? Are there any features that aren't needed?

In [ ]:
ads.corr()
#Yes. (isbuyer and buy_freq) and (last_buy and last_visit) can be considered a redundancy